<a href="https://colab.research.google.com/github/L1KASA/DL_Course/blob/main/lab_4/6409_%D0%A2%D0%B0%D1%85%D0%BC%D0%B0%D0%B7%D0%BE%D0%B2%D0%B0%D0%90%D0%A8_%D0%9B%D0%A04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа 4

Tensorflow 2.x

1) Подготовка данных

2) Использование Keras Model API

3) Использование Keras Sequential + Functional API

https://www.tensorflow.org/tutorials

Для выполнения лабораторной работы необходимо установить tensorflow версии 2.0 или выше .

Рекомендуется использовать возможности Colab'а по обучению моделей на GPU.



In [1]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы.

In [2]:
def load_data(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    data = tf.keras.datasets.mnist.load_data()
    (X_train, y_train), (X_test, y_test) = data
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 28, 28)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 28, 28)
Validation labels shape:  (1000,)
Test data shape:  (10000, 28, 28)
Test labels shape:  (10000,)


In [3]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y

        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


X_train, y_train, X_val, y_val, X_test, y_test = load_data()
X_train = X_train.reshape(-1, 28, 28, 1)
X_val = X_val.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [4]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 28, 28, 1) (64,)
1 (64, 28, 28, 1) (64,)
2 (64, 28, 28, 1) (64,)
3 (64, 28, 28, 1) (64,)
4 (64, 28, 28, 1) (64,)
5 (64, 28, 28, 1) (64,)
6 (64, 28, 28, 1) (64,)


#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети.

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [5]:
device = '/device:GPU:0'

class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()

    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    with tf.device(device):
        scores = model(x)
        print(scores.shape)

test_TwoLayerFC()

(64, 10)


/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Реализуйте трехслойную CNN для вашей задачи классификации.

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU
5. Полносвязный слой
6. Функция активации Softmax

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [6]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        self.conv1 = tf.keras.layers.Conv2D(channel_1, kernel_size=(5, 5), strides=(1, 1), padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(channel_2, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################

    def call(self, x, training=False):
        scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        scores = self.fc(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        return scores

In [7]:
def test_ThreeLayerConvNet():
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [8]:
print_every = 200

def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.

    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for

    Returns: Nothing, but prints progress during trainingn
    """
    with tf.device(device):


        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

        model = model_init_fn()
        optimizer = optimizer_init_fn()

        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

        t = 0
        for epoch in range(num_epochs):

            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_states()
            train_accuracy.reset_states()

            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:

                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)

                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)

                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)

                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [9]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.9707770347595215, Accuracy: 7.8125, Val Loss: 2.5538721084594727, Val Accuracy: 23.700000762939453
Iteration 200, Epoch 1, Loss: 0.5073060393333435, Accuracy: 85.09017181396484, Val Loss: 0.44869092106819153, Val Accuracy: 86.29999542236328
Iteration 400, Epoch 1, Loss: 0.4065001606941223, Accuracy: 88.00654602050781, Val Loss: 0.36424949765205383, Val Accuracy: 89.5
Iteration 600, Epoch 1, Loss: 0.35976237058639526, Accuracy: 89.4186782836914, Val Loss: 0.3213493525981903, Val Accuracy: 91.0999984741211


Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 .

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50% .

In [10]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate, 0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.320552349090576, Accuracy: 9.375, Val Loss: 2.292663335800171, Val Accuracy: 12.800000190734863
Iteration 200, Epoch 1, Loss: 0.5310657024383545, Accuracy: 84.38277435302734, Val Loss: 0.4360634982585907, Val Accuracy: 86.69999694824219
Iteration 400, Epoch 1, Loss: 0.3846120834350586, Accuracy: 88.63388061523438, Val Loss: 0.24996791779994965, Val Accuracy: 92.9000015258789
Iteration 600, Epoch 1, Loss: 0.31245699524879456, Accuracy: 90.80699157714844, Val Loss: 0.2337530106306076, Val Accuracy: 92.69999694824219


# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [11]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (28, 28, 1)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax',
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.3092200756073, Accuracy: 9.375, Val Loss: 2.8534798622131348, Val Accuracy: 19.30000114440918
Iteration 200, Epoch 1, Loss: 0.5113052129745483, Accuracy: 85.01243591308594, Val Loss: 0.4549221992492676, Val Accuracy: 85.69999694824219
Iteration 400, Epoch 1, Loss: 0.40786993503570557, Accuracy: 87.97148132324219, Val Loss: 0.36683934926986694, Val Accuracy: 89.60000610351562
Iteration 600, Epoch 1, Loss: 0.3615171015262604, Accuracy: 89.38227844238281, Val Loss: 0.3305671811103821, Val Accuracy: 90.9000015258789


Альтернативный менее гибкий способ обучения:

In [12]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

313/313 [==============================] - 3s 10ms/step - loss: 0.2239 - sparse_categorical_accuracy: 0.9350


[0.22392013669013977, 0.9350000023841858]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [13]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(28, (5, 5), padding='same', activation='relu', kernel_initializer=tf.initializers.VarianceScaling(scale=2.0)),
        tf.keras.layers.Conv2D(14, (3, 3), padding='same', activation='relu', kernel_initializer=tf.initializers.VarianceScaling(scale=2.0)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation='softmax', kernel_initializer=tf.initializers.VarianceScaling(scale=2.0))
    ])

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate, momentum=0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.9222183227539062, Accuracy: 12.5, Val Loss: 2.809948682785034, Val Accuracy: 11.90000057220459
Iteration 200, Epoch 1, Loss: 0.5595692992210388, Accuracy: 83.0768051147461, Val Loss: 0.4951278865337372, Val Accuracy: 84.30000305175781
Iteration 400, Epoch 1, Loss: 0.42088350653648376, Accuracy: 87.25841522216797, Val Loss: 0.32914459705352783, Val Accuracy: 91.0999984741211
Iteration 600, Epoch 1, Loss: 0.35670727491378784, Accuracy: 89.33547973632812, Val Loss: 0.2865014970302582, Val Accuracy: 91.19999694824219


In [14]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

313/313 [==============================] - 4s 14ms/step - loss: 0.1162 - sparse_categorical_accuracy: 0.9636


[0.11618942767381668, 0.9635999798774719]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры.

Ниже представлен пример для полносвязной сети.

In [15]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)

    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)

    with tf.device(device):
        scores = model(x)
        print(scores.shape)

test_two_layer_fc_functional()

(64, 10)


In [16]:
input_shape = (28, 28, 1)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.1220805644989014, Accuracy: 3.125, Val Loss: 2.8726797103881836, Val Accuracy: 27.5
Iteration 200, Epoch 1, Loss: 0.5130127668380737, Accuracy: 85.11349487304688, Val Loss: 0.4344344735145569, Val Accuracy: 86.4000015258789
Iteration 400, Epoch 1, Loss: 0.4109102487564087, Accuracy: 88.0377197265625, Val Loss: 0.3605281412601471, Val Accuracy: 89.20000457763672
Iteration 600, Epoch 1, Loss: 0.3633130192756653, Accuracy: 89.40567779541016, Val Loss: 0.33132171630859375, Val Accuracy: 90.10000610351562


Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут).

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [20]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        # Определение сверточных слоев с активацией ReLU
        self.conv1 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.max_pool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))

        # Определение слоев полносвязной части сети
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(256, activation='relu')
        self.fc2 = tf.keras.layers.Dense(10, activation='softmax')

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################

    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        # Применение сверточных слоев
        x = self.conv1(input_tensor)
        x = self.conv2(x)
        x = self.max_pool(x)

        # Применение полносвязных слоев
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x



print_every = 700
num_epochs = 10

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate)

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 2.344405174255371, Accuracy: 9.375, Val Loss: 2.710981845855713, Val Accuracy: 23.700000762939453
Iteration 700, Epoch 1, Loss: 0.13505315780639648, Accuracy: 95.87865447998047, Val Loss: 0.0880117267370224, Val Accuracy: 97.29999542236328
Iteration 1400, Epoch 2, Loss: 0.038813211023807526, Accuracy: 98.8090591430664, Val Loss: 0.07429854571819305, Val Accuracy: 97.5
Iteration 2100, Epoch 3, Loss: 0.020551087334752083, Accuracy: 99.34369659423828, Val Loss: 0.10064608603715897, Val Accuracy: 97.69999694824219
Iteration 2800, Epoch 4, Loss: 0.011912074871361256, Accuracy: 99.63655090332031, Val Loss: 0.10644587129354477, Val Accuracy: 97.29999542236328
Iteration 3500, Epoch 5, Loss: 0.009552106261253357, Accuracy: 99.6531753540039, Val Loss: 0.049508191645145416, Val Accuracy: 98.5999984741211
Iteration 4200, Epoch 6, Loss: 0.009121929295361042, Accuracy: 99.65885925292969, Val Loss: 0.10220829397439957, Val Accuracy: 97.89999389648438
Iteration 4900, Epoch 

In [21]:
def model_init_fn():
    return CustomConvNet()

# Let's try SGD optimizer
def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.SGD(learning_rate)

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 2.2825896739959717, Accuracy: 25.0, Val Loss: 2.2941088676452637, Val Accuracy: 18.099998474121094
Iteration 700, Epoch 1, Loss: 1.2664738893508911, Accuracy: 69.70622253417969, Val Loss: 0.5560643672943115, Val Accuracy: 84.89999389648438
Iteration 1400, Epoch 2, Loss: 0.37186968326568604, Accuracy: 89.51771545410156, Val Loss: 0.4126312732696533, Val Accuracy: 87.69999694824219
Iteration 2100, Epoch 3, Loss: 0.2969549298286438, Accuracy: 91.4625473022461, Val Loss: 0.369508296251297, Val Accuracy: 89.70000457763672
Iteration 2800, Epoch 4, Loss: 0.2622646987438202, Accuracy: 92.49503326416016, Val Loss: 0.3321898281574249, Val Accuracy: 90.0
Iteration 3500, Epoch 5, Loss: 0.2319941222667694, Accuracy: 93.27803039550781, Val Loss: 0.30583885312080383, Val Accuracy: 90.9000015258789
Iteration 4200, Epoch 6, Loss: 0.2103988528251648, Accuracy: 93.90161895751953, Val Loss: 0.28569164872169495, Val Accuracy: 91.19999694824219
Iteration 4900, Epoch 7, Loss: 0.19

In [28]:
class CustomConvNet_drp(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet_drp, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        # Определение сверточных слоев
        self.conv1 = tf.keras.layers.Conv2D(32, (3, 3), padding='same')
        self.conv2 = tf.keras.layers.Conv2D(16, (3, 3), padding='same')

        # Определение функций активации и слоев dropout
        self.relu1 = tf.keras.layers.ReLU()
        self.dropout1 = tf.keras.layers.Dropout(0.5)
        self.relu2 = tf.keras.layers.ReLU()
        self.dropout2 = tf.keras.layers.Dropout(0.5)

        # Определение слоев полносвязной части сети
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(10, activation='softmax')

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################

    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        # Применение сверточных слоев
        x = self.conv1(input_tensor)
        x = self.relu1(x)
        x = self.dropout1(x, training=training)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.dropout2(x, training=training)

        # Применение полносвязных слоев
        x = self.flatten(x)
        x = self.dense(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


In [29]:
def model_init_fn():
    return CustomConvNet_drp()

# Let's try Adam optimizer and another architecture
def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate)

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 2.317478656768799, Accuracy: 7.8125, Val Loss: 2.265691041946411, Val Accuracy: 19.69999885559082
Iteration 700, Epoch 1, Loss: 0.25534796714782715, Accuracy: 92.22315979003906, Val Loss: 0.14222152531147003, Val Accuracy: 95.20000457763672
Iteration 1400, Epoch 2, Loss: 0.0959387868642807, Accuracy: 96.96604919433594, Val Loss: 0.109047070145607, Val Accuracy: 96.9000015258789
Iteration 2100, Epoch 3, Loss: 0.07766222208738327, Accuracy: 97.6329116821289, Val Loss: 0.11098338663578033, Val Accuracy: 96.80000305175781
Iteration 2800, Epoch 4, Loss: 0.06602685153484344, Accuracy: 97.84107971191406, Val Loss: 0.09250353276729584, Val Accuracy: 97.0
Iteration 3500, Epoch 5, Loss: 0.05619821697473526, Accuracy: 98.16218566894531, Val Loss: 0.08318503201007843, Val Accuracy: 97.39999389648438
Iteration 4200, Epoch 6, Loss: 0.05103704705834389, Accuracy: 98.29009246826172, Val Loss: 0.07944213598966599, Val Accuracy: 97.39999389648438
Iteration 4900, Epoch 7, Loss

In [30]:
def model_init_fn():
    return CustomConvNet_drp()

# Let's try Adam optimizer and another architecture
def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.SGD(learning_rate)

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 2.2921738624572754, Accuracy: 10.9375, Val Loss: 2.3128442764282227, Val Accuracy: 5.900000095367432
Iteration 700, Epoch 1, Loss: 1.7358227968215942, Accuracy: 42.0938835144043, Val Loss: 0.6991236209869385, Val Accuracy: 79.5999984741211
Iteration 1400, Epoch 2, Loss: 0.5848836302757263, Accuracy: 81.6904525756836, Val Loss: 0.4899598956108093, Val Accuracy: 86.19999694824219
Iteration 2100, Epoch 3, Loss: 0.48626258969306946, Accuracy: 85.19332122802734, Val Loss: 0.45410043001174927, Val Accuracy: 87.30000305175781
Iteration 2800, Epoch 4, Loss: 0.4456319212913513, Accuracy: 86.61468505859375, Val Loss: 0.42748984694480896, Val Accuracy: 88.0
Iteration 3500, Epoch 5, Loss: 0.4155360460281372, Accuracy: 87.82537078857422, Val Loss: 0.40877705812454224, Val Accuracy: 89.0
Iteration 4200, Epoch 6, Loss: 0.4034092426300049, Accuracy: 88.11910247802734, Val Loss: 0.3953026235103607, Val Accuracy: 89.20000457763672
Iteration 4900, Epoch 7, Loss: 0.385917931795

Опишите все эксперименты, результаты. Сделайте выводы.

Основные выводы по проделанной работе:
1. Лучше всего данные работают с моделью без Dropout слоев как с Adam, так и с SGD. Но даже так все эксперименты показали отличные результаты.
2. В нашем случае наиболее эффективным оптимизатором оказался Adam.